<a href="https://colab.research.google.com/github/Dhanush-adk/intro_to_dl/blob/main/Assignment_4/Assignment_4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import random

In [50]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [51]:
english_to_french = [
    ("I am cold", "J'ai froid"),

    ("You are tired", "Tu es fatigué"),

    ("He is hungry", "Il a faim"),

    ("She is happy", "Elle est heureuse"),

    ("We are friends", "Nous sommes amis"),

    ("They are students", "Ils sont étudiants"),

    ("The cat is sleeping", "Le chat dort"),

    ("The sun is shining", "Le soleil brille"),

    ("We love music", "Nous aimons la musique"),

    ("She speaks French fluently", "Elle parle français couramment"),

    ("He enjoys reading books", "Il aime lire des livres"),

    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),

    ("The movie starts at 7 PM", "Le film commence à 19 heures"),

    ("She wears a red dress", "Elle porte une robe rouge"),

    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),

    ("He drives a blue car", "Il conduit une voiture bleue"),

    ("They visit museums often", "Ils visitent souvent des musées"),

    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),

    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),

    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),

    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),

    ("They travel around the world", "Ils voyagent autour du monde"),

    ("The book is on the table", "Le livre est sur la table"),

    ("She dances gracefully", "Elle danse avec grâce"),

    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),

    ("He works hard every day", "Il travaille dur tous les jours"),

    ("They speak different languages", "Ils parlent différentes langues"),

    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),

    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),

    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),

    ("The dog barks loudly", "Le chien aboie bruyamment"),

    ("He sings beautifully", "Il chante magnifiquement"),

    ("They swim in the pool", "Ils nagent dans la piscine"),

    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),

    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),

    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),

    ("He paints landscapes", "Il peint des paysages"),

    ("They laugh at the joke", "Ils rient de la blague"),

    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),

    ("She runs in the park", "Elle court dans le parc"),

    ("We travel by train", "Nous voyageons en train"),

    ("He writes a letter", "Il écrit une lettre"),

    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),

    ("The baby cries", "Le bébé pleure"),

    ("She studies hard for exams", "Elle étudie dur pour les examens"),

    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),

    ("He fixes the car", "Il répare la voiture"),

    ("They drink coffee in the morning", "Ils boivent du café le matin"),

    ("The sun sets in the evening", "Le soleil se couche le soir"),

    ("She dances at the party", "Elle danse à la fête"),

    ("We play music at the concert", "Nous jouons de la musique au concert"),

    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),

    ("They study French grammar", "Ils étudient la grammaire française"),

    ("The rain falls gently", "La pluie tombe doucement"),

    ("She sings a song", "Elle chante une chanson"),

    ("We watch a movie together", "Nous regardons un film ensemble"),

    ("He sleeps deeply", "Il dort profondément"),

    ("They travel to Paris", "Ils voyagent à Paris"),

    ("The children play in the park", "Les enfants jouent dans le parc"),

    ("She walks along the beach", "Elle se promène le long de la plage"),

    ("We talk on the phone", "Nous parlons au téléphone"),

    ("He waits for the bus", "Il attend le bus"),

    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),

    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),

    ("She dreams of flying", "Elle rêve de voler"),

    ("We work in the office", "Nous travaillons au bureau"),

    ("He studies history", "Il étudie l'histoire"),

    ("They listen to the radio", "Ils écoutent la radio"),

    ("The wind blows gently", "Le vent souffle doucement"),

    ("She swims in the ocean", "Elle nage dans l'océan"),

    ("We dance at the wedding", "Nous dansons au mariage"),

    ("He climbs the mountain", "Il gravit la montagne"),

    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),

    ("The cat meows loudly", "Le chat miaule bruyamment"),

    ("She paints a picture", "Elle peint un tableau"),

    ("We build a sandcastle", "Nous construisons un château de sable"),

    ("He sings in the choir", "Il chante dans le chœur")
]

In [52]:
SOS_token = 0  # Start Of Sequence Token
EOS_token = 1  # End Of Sequence Token
max_length = 20

char_to_index_Eng = word_to_index = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_Eng = {i: char for char, i in char_to_index_Eng.items()}

char_to_index_French = {"SOS": 0, "EOS": 1, **{word: i+2 for i, word in enumerate(sorted(set(word for pair in english_to_french for sentence in pair for word in sentence.split())))}}
index_to_char_French = {i: char for char, i in char_to_index_French.items()}

class EngFrDataset(Dataset):
    """Custom Dataset class for handling English-French pairs."""
    def __init__(self, dataset, char_to_index_Eng, char_to_index_French):
        self.dataset = dataset
        self.char_to_index_Eng = char_to_index_Eng
        self.char_to_index_French = char_to_index_French

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        English_sentence, French_sentence = self.dataset[idx]
        English_tensor = torch.tensor([self.char_to_index_Eng[word] for word in English_sentence.split()] + [EOS_token], dtype=torch.long)
        French_tensor = torch.tensor([self.char_to_index_French[word] for word in French_sentence.split()] + [EOS_token], dtype=torch.long)
        return English_tensor, French_tensor

class Encoder(nn.Module):
    """Encoder that processes the input sequence and returns its hidden states."""
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()  # Corrected super() call
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device))


class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=20):
        super(AttnDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = torch.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = torch.relu(output)
        output, hidden = self.gru(output, hidden)

        output = torch.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Training loop
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length = 20):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()
        #Calculate Loss
        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def evaluate_and_show_examples(encoder, decoder, dataloader, criterion, n_examples = 0):
    encoder.eval()
    decoder.eval()

    total_loss = 0
    total_sentences = 0
    correct_predictions = 0

    with torch.no_grad():
        for i, (input_tensor_English, target_tensor_French) in enumerate(dataloader):
            input_tensor_English = input_tensor_English[0].to(device)
            target_tensor_French = target_tensor_French[0].to(device)

            encoder_hidden = encoder.initHidden()


            input_length = input_tensor_English.size(0)
            target_length = target_tensor_French.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor_English[ei].unsqueeze(0), encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor_French[di].unsqueeze(0))
                if decoder_input.item() == EOS_token:
                    break
            # Calculate and print loss and accuracy for the evaluation
            total_loss += loss.item() / target_length
            total_sentences += 1

            if predicted_indices == target_tensor_French.tolist():
                correct_predictions += 1

            if n_examples > 0:
            # Optionally, print some examples
              if i < n_examples:
                  predicted_string = ' '.join([index_to_char_French[index] for index in predicted_indices if index not in (SOS_token, EOS_token)])
                  target_string = ' '.join([index_to_char_French[index.item()] for index in target_tensor_French if index.item() not in (SOS_token, EOS_token)])
                  input_string = ' '.join([index_to_char_Eng[index.item()] for index in input_tensor_English if index.item() not in (SOS_token, EOS_token)])

                  print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')
        # Print overall evaluation results
        average_loss = total_loss / len(dataloader)
        accuracy = correct_predictions / total_sentences
        # print(f'Evaluation Loss: {average_loss}, Accuracy: {accuracy}')
        return average_loss, accuracy

In [53]:
dataset = EngFrDataset(english_to_french, char_to_index_Eng, char_to_index_French)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

input_size=len(char_to_index_Eng)
hidden_size=256
output_size=len(char_to_index_French)

criterion = nn.NLLLoss()
encoder_eng = Encoder(input_size=input_size, hidden_size=hidden_size).to(device)
decoder_french = AttnDecoder(hidden_size=hidden_size, output_size=output_size).to(device)
encoder_optimizer = optim.SGD(encoder_eng.parameters(), lr = 0.01)
decoder_optimizer = optim.SGD(decoder_french.parameters(), lr = 0.01)

n_epochs = 100
# # Perform evaluation
for epoch in range(n_epochs):
    encoder_eng.train()  # Set encoder to training mode
    decoder_french.train()  # Set decoder to training mode

    total_loss = 0
    for input_tensor_English, target_tensor_French in dataloader:
        input_tensor_English = input_tensor_English[0].to(device)
        target_tensor_French = target_tensor_French[0].to(device)

        loss = train(input_tensor_English, target_tensor_French, encoder_eng, decoder_french, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

    # Before evaluating, switch to evaluation mode
    encoder_eng.eval()  # Set encoder to evaluation mode
    decoder_french.eval()  # Set decoder to evaluation mode
    eval_loss, eval_accuracy = evaluate_and_show_examples(encoder_eng, decoder_french, dataloader, criterion)

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}, Eval Loss: {eval_loss}, Eval Accuracy: {eval_accuracy}')


Epoch 10, Loss: 2.738121585844964, Eval Loss: 2.3216656118686663, Eval Accuracy: 0.0
Epoch 20, Loss: 1.4505679161370209, Eval Loss: 1.3407914777486205, Eval Accuracy: 0.09090909090909091
Epoch 30, Loss: 0.4548405049548052, Eval Loss: 0.35640997505297345, Eval Accuracy: 0.7792207792207793
Epoch 40, Loss: 0.12103052600616158, Eval Loss: 0.11452964418292219, Eval Accuracy: 0.961038961038961
Epoch 50, Loss: 0.05023821594450455, Eval Loss: 0.04444435516856513, Eval Accuracy: 1.0
Epoch 60, Loss: 0.03062241718924075, Eval Loss: 0.027900673224032238, Eval Accuracy: 1.0
Epoch 70, Loss: 0.022042523398020118, Eval Loss: 0.020295343607037466, Eval Accuracy: 1.0
Epoch 80, Loss: 0.017278723032015046, Eval Loss: 0.015933790809137562, Eval Accuracy: 1.0
Epoch 90, Loss: 0.01416808415317174, Eval Loss: 0.013098249823305015, Eval Accuracy: 1.0
Epoch 100, Loss: 0.011921801424478346, Eval Loss: 0.011100618777767062, Eval Accuracy: 1.0


In [55]:
evaluate_and_show_examples(encoder_eng, decoder_french, dataloader, criterion, n_examples = 5)

Input: She walks along the beach, Target: Elle se promène le long de la plage, Predicted: Elle se promène le long de la plage
Input: We dance at the wedding, Target: Nous dansons au mariage, Predicted: Nous dansons au mariage
Input: We eat breakfast together, Target: Nous prenons le petit déjeuner ensemble, Predicted: Nous prenons le petit déjeuner ensemble
Input: You are tired, Target: Tu es fatigué, Predicted: Tu es fatigué
Input: They hike in the forest, Target: Ils font de la randonnée dans la forêt, Predicted: Ils font de la randonnée dans la forêt


(0.01110061877776706, 1.0)